In [1]:
# =============================================================================
# BigQuery Setup & Authentication
# =============================================================================

from google.cloud import bigquery
import pydata_google_auth
import pandas as pd
from datetime import datetime

# Set your project ID (required for BigQuery queries)
# Replace with your Google Cloud project ID
PROJECT_ID = "1044108980210"  # <-- UPDATE THIS if needed

# Authenticate with Google Cloud (opens browser for login)
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
    use_local_webserver=True
)

# Create BigQuery client with credentials
client = bigquery.Client(project=PROJECT_ID, credentials=credentials)

# Load the BigQuery magic extension
%load_ext google.cloud.bigquery

print(f"✓ Authenticated successfully")
print(f"✓ BigQuery client created")
print(f"✓ Project ID: {PROJECT_ID}")

✓ Authenticated successfully
✓ BigQuery client created
✓ Project ID: 1044108980210


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [2]:
# =============================================================================
# FULLY IN-BIGQUERY Pipeline: No Local Upload Required
# =============================================================================
# This approach keeps EVERYTHING in BigQuery:
# 1. Query attacks directly INTO a BigQuery table (not local)
# 2. Join attacks table with token_transfers (both in BigQuery)
# 3. Download only the final results
#
# MUCH faster for large datasets - no upload bottleneck!

import uuid

# Dataset for storing our tables - ARBITRUM SPECIFIC
BQ_DATASET = "arbitrum_vlad_query"

def setup_bigquery_dataset():
    """Create the BigQuery dataset if it doesn't exist."""
    dataset_id = f"{PROJECT_ID}.{BQ_DATASET}"
    try:
        client.get_dataset(dataset_id)
        print(f"✓ Dataset exists: {dataset_id}")
    except:
        print(f"📁 Creating dataset: {dataset_id}")
        dataset = bigquery.Dataset(dataset_id)
        dataset.location = "US"
        client.create_dataset(dataset, exists_ok=True)
        print(f"✓ Created dataset: {dataset_id}")
    return dataset_id

def drop_all_analysis_tables():
    """
    ⚠️  WARNING: This function PERMANENTLY DELETES all analysis tables in the dataset.
    
    Drops all tables used in the Arbitrum poisoning detection analysis:
    - identified_attacks
    - verified_attacks
    - successful_attacks
    
    This allows you to recollect data with updated logic or parameters.
    
    ⚠️  THIS ACTION CANNOT BE UNDONE - All data in these tables will be lost!
    
    Use this when you've made changes to the detection logic and need fresh results.
    """
    
    dataset_id = f"{PROJECT_ID}.{BQ_DATASET}"
    
    print("\n" + "=" * 70)
    print("⚠️  WARNING: PERMANENT DATA DELETION")
    print("=" * 70)
    print(f"This will DELETE all analysis tables in: {dataset_id}")
    print("\nTables that will be dropped:")
    print("  - identified_attacks")
    print("  - verified_attacks")
    print("  - successful_attacks")
    print("\n⚠️  THIS ACTION CANNOT BE UNDONE - All data will be permanently deleted!")
    print("=" * 70)
    
    # Prompt user for confirmation
    confirmation = input("\nType 'DELETE_ALL_TABLES' to confirm, or press Enter to cancel: ").strip()
    
    if confirmation != "DELETE_ALL_TABLES":
        print("\n✗ Deletion cancelled. No tables were deleted.")
        return
    
    # List of tables to drop
    tables_to_drop = [
        "identified_attacks",
        "verified_attacks", 
        "successful_attacks"
    ]
    
    dropped_count = 0
    failed_count = 0
    
    print("\n" + "=" * 70)
    print("DROPPING TABLES...")
    print("=" * 70)
    
    for table_name in tables_to_drop:
        try:
            table_id = f"{dataset_id}.{table_name}"
            client.delete_table(table_id, not_found_ok=True)
            print(f"✓ Dropped: {table_name}")
            dropped_count += 1
        except Exception as e:
            print(f"✗ Failed to drop {table_name}: {e}")
            failed_count += 1
    
    print("\n" + "=" * 70)
    print("DELETION COMPLETE")
    print("=" * 70)
    print(f"✓ Successfully dropped: {dropped_count} table(s)")
    if failed_count > 0:
        print(f"✗ Failed to drop: {failed_count} table(s)")
    print(f"\nYou can now re-run the pipeline to recollect data with updated logic.")
    print("=" * 70 + "\n")




print("✓ In-BigQuery pipeline functions loaded")
print(f"✓ Dataset: {BQ_DATASET}")
print("✓ drop_all_analysis_tables() available for data reset")
print("✓ Now using bigquery-public-data.goog_blockchain_arbitrum_one_us.logs")

✓ In-BigQuery pipeline functions loaded
✓ Dataset: arbitrum_vlad_query
✓ drop_all_analysis_tables() available for data reset
✓ Now using bigquery-public-data.goog_blockchain_arbitrum_one_us.logs


In [3]:
# =============================================================================
# Final Arbitrum Query
# =============================================================================
# This version only identifies TRUE poisoning attacks where:
# 1. Attacker address matches BOTH prefix AND suffix of a legitimate address victim used
# 2. Captures the "intended address" being impersonated
# 3. Filters out perfect matches (where attacker == intended address entirely)
# Uses logs table for Arbitrum.

def query_verified_attacks_to_bigquery(
    start_date: str,
    end_date: str,
    table_name: str = "verified_attacks",
    append: bool = False
):
    """
    Query attacks with prefix/suffix validation and intended address.
    Only flags transfers as attacks if attacker address matches 
    BOTH the prefix AND suffix of an address the victim previously sent to.
    Uses logs table for Arbitrum.
    
    Lookback is in BLOCKS (4600 blocks ≈ 20 minutes on Arbitrum).
    min_victim_transfer_usd: Minimum value of victim's prior transfer to count as "intended address".
                             Set to 0 to consider ALL victim transfers as potential intended addresses.
    
    Deduplication Strategy: When multiple historical transfers match the same attack,
    keeps the transfer that is CLOSEST in time (temporal proximity) to the attack,
    as this is most likely the address the attacker observed the victim sending to.
    
    Perfect Match Filtering: Removes any cases where attacker_address == intended_address
    for all 40 hex characters, as these are not poisoning attacks but legitimate transfers.
    """
    print("=" * 60)
    print("VERIFIED ATTACK DETECTION (with prefix/suffix matching)")
    print("=" * 60)
    print(f"Date range: {start_date} to {end_date}")
    print(f"Deduplication: Closest temporal match (most recent before attack)")
    print(f"Filters: Removes perfect matches (full 40-char address match)")
    print(f"Data source: Arbitrum logs table")
    print("=" * 60)
    
    setup_bigquery_dataset()
    dest_table = f"{PROJECT_ID}.{BQ_DATASET}.{table_name}"
    
    # For the date filter, use a larger buffer to ensure we capture victim history
    # at month boundaries. Use 7 days to ensure we capture previous month's data
    # even if attacks happen on the first day of a new month.
    lookback_date = (pd.to_datetime(start_date) - pd.Timedelta(days=7)).strftime('%Y-%m-%d')
    
    query = f"""
    -- 1. CONFIGURATION
    DECLARE analysis_start TIMESTAMP DEFAULT TIMESTAMP("{start_date}");
    DECLARE analysis_end   TIMESTAMP DEFAULT TIMESTAMP("{end_date}");

    -- Logic Settings
    DECLARE lookahead_blocks INT64 DEFAULT 4600; 
    DECLARE prefix_len INT64 DEFAULT 3;
    DECLARE suffix_len INT64 DEFAULT 4;
    DECLARE dust_threshold FLOAT64 DEFAULT 1.0; -- Upper limit for a "Tiny" transfer ($1.00)

    DECLARE start_block DEFAULT (
    SELECT MIN(block_number)
    FROM `bigquery-public-data.goog_blockchain_arbitrum_one_us.logs`
    WHERE block_timestamp >= analysis_start
    );

    DECLARE end_block DEFAULT (
    SELECT MAX(block_number)
    FROM `bigquery-public-data.goog_blockchain_arbitrum_one_us.logs`
    WHERE block_timestamp < analysis_end
    );

    CREATE TEMP FUNCTION HexToBigNumeric(hex STRING)
    RETURNS BIGNUMERIC
    LANGUAGE js AS \"\"\"
    // Remove 0x prefix if present
    hex = hex.startsWith("0x") ? hex.slice(2) : hex;

    // Convert hex to BigInt
    const value = BigInt('0x' + hex);

    // Return as string so BigQuery can cast it
    return value.toString();
    \"\"\";

    -- 2. CREATE OR INSERT INTO TABLE
    {'CREATE OR REPLACE TABLE `' + dest_table + '` AS' if not append else 'INSERT INTO `' + dest_table + '`'}

    -- 3. DATA EXTRACTION
    WITH raw_transfers AS (
        SELECT
        block_number,
        block_timestamp,
        transaction_hash,
        address AS token_address,
        -- Decode from & to (topics[1] and topics[2])
        CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(1)], 27)) AS from_addr,
        CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(2)], 27)) AS to_addr,
        -- cast from amounts
        HexToBigNumeric(data) AS value,
        HexToBigNumeric(data) / 1e6 AS value_usd,
        -- add token symbol manually
        -- CASE 
        --     WHEN LOWER(address) = usdc_native  THEN "USDC.e"
        --     WHEN LOWER(address) = usdc_bridged THEN "USDC"
        --     WHEN LOWER(address) = usdt_addr    THEN "USDT"
        -- END AS token_symbol
        FROM `bigquery-public-data.goog_blockchain_arbitrum_one_us.logs`
        WHERE block_number BETWEEN start_block AND end_block
        AND removed = FALSE
        AND topics[SAFE_OFFSET(0)] = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
        AND address IN (
            "0xaf88d065e77c8cc2239327c5edb3a432268e5831", -- USDC Native
            "0xff970a61a04b1ca14834a43f5de4533ebddb5cc8", -- USDC Bridged
            "0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9"  -- USDT
        )
    ),

    -- 4. IDENTIFY PAIRS
    attack_pairs AS (
        SELECT
            S.transaction_hash AS attack_tx_hash,
            S.block_number AS attack_block,
            S.block_timestamp AS attack_ts,
            -- S.token_symbol,
            S.value_usd,
            
            -- NEW COLUMN: Attack Type
            CASE 
                WHEN S.value_usd = 0 THEN 'zero_value'
                ELSE 'tiny_transfer'
            END AS attack_type,

            -- Attacker Labeling
            CASE 
                WHEN S.from_addr = V.from_addr THEN S.to_addr 
                ELSE S.from_addr 
            END AS attacker_address,

            V.from_addr AS victim_address,
            V.to_addr AS intended_address,
            V.transaction_hash AS intended_tx_hash,
            V.block_timestamp AS intended_ts,
            V.block_number AS intended_block,
            
            CASE WHEN S.from_addr = V.from_addr THEN 'spoof_outgoing' ELSE 'spam_incoming' END AS direction

        FROM raw_transfers AS V -- History
        JOIN raw_transfers AS S -- Attack
            ON S.token_address = V.token_address
            AND S.block_number >= V.block_number
            AND S.block_number <= (V.block_number + lookahead_blocks)
        WHERE
            V.value_usd > 0
            AND V.block_timestamp BETWEEN analysis_start AND analysis_end
            
            -- MODIFIED FILTER: Allow 0 OR Small Values
            AND S.value_usd <= dust_threshold
            
            AND (
                (S.from_addr = V.from_addr)
                OR
                (S.to_addr = V.from_addr)
            )
            AND S.transaction_hash != V.transaction_hash
    )

    -- 5. FINAL SELECTION
    SELECT 
        attack_tx_hash,
        attack_block,
        attack_ts,
        -- token_symbol,
        attack_type, -- <--- Included in final output
        value_usd,   -- <--- Good to verify the actual amount
        direction,
        attacker_address,
        victim_address,
        intended_address,
        intended_ts,
        (attack_block - intended_block) AS blocks_delay
    FROM attack_pairs
    WHERE
        SUBSTR(attacker_address, 3, prefix_len) = SUBSTR(intended_address, 3, prefix_len)
        AND
        SUBSTR(attacker_address, -1 * suffix_len) = SUBSTR(intended_address, -1 * suffix_len)
        AND attacker_address != intended_address

    -- DEDUPLICATION
    QUALIFY ROW_NUMBER() OVER(
        PARTITION BY attack_tx_hash, attacker_address, victim_address, direction
        ORDER BY intended_ts DESC
    ) = 1
    ORDER BY attack_ts, attack_tx_hash
    """
    
    # DON'T set destination in job_config - the SQL script handles it
    job_config = bigquery.QueryJobConfig()
    print(f"\n🔍 Running verified attack detection...")
    print(f"📝 Destination: {dest_table}")
    
    job = client.query(query, job_config=job_config)
    job.result()
    
    table = client.get_table(dest_table)
    print(f"\n✓ Query complete!")
    print(f"✓ Verified attacks found: {table.num_rows:,}")
    print(f"✓ Table: {dest_table}")
    
    return dest_table

print("✓ query_verified_attacks_to_bigquery() loaded")
print("  - Data source: bigquery-public-data.goog_blockchain_arbitrum_one_us.logs")
print("  - Only flags attacks where BOTH prefix AND suffix match")
print("  - Uses block-based lookback (~20 minutes = 4600 blocks)")
print("  - Considers ALL victim transfers as potential intended addresses")
print("  - Deduplicates: Closest temporal match (most recent before attack)")
print("  - Filters out perfect matches (full 40-char address matches)")
print("  - Includes intended_address in output")

✓ query_verified_attacks_to_bigquery() loaded
  - Data source: bigquery-public-data.goog_blockchain_arbitrum_one_us.logs
  - Only flags attacks where BOTH prefix AND suffix match
  - Uses block-based lookback (~20 minutes = 4600 blocks)
  - Considers ALL victim transfers as potential intended addresses
  - Deduplicates: Closest temporal match (most recent before attack)
  - Filters out perfect matches (full 40-char address matches)
  - Includes intended_address in output


In [ ]:
# =============================================================================
# FULL PIPELINE: Verified Address Poisoning Detection (Batch by Time Period)
# =============================================================================
# This pipeline detects TRUE address poisoning attacks using prefix/suffix matching.
# Processes data in configurable time chunks to avoid timeouts and manage costs.
#
# All results are appended to a single BigQuery table.

import time
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_month_ranges(year: int) -> list:
    """
    Generate list of (start_date, end_date) tuples for each month in a year.
    
    Args:
        year: The year to generate months for (e.g., 2023)
    
    Returns:
        List of tuples: [("2023-01-01", "2023-02-01"), ("2023-02-01", "2023-03-01"), ...]
    """
    months = []
    for month in range(1, 13):
        start = f"{year}-{month:02d}-01"
        if month == 12:
            end = f"{year + 1}-01-01"
        else:
            end = f"{year}-{month + 1:02d}-01"
        months.append((start, end))
    return months


def get_month_ranges_between(start_date: str, end_date: str) -> list:
    """
    Generate list of (start_date, end_date) tuples for each month between two dates.
    
    Args:
        start_date: Start date 'YYYY-MM-DD'
        end_date: End date 'YYYY-MM-DD' (exclusive)
    
    Returns:
        List of tuples for each month in the range
    """
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    months = []
    current = start.replace(day=1)
    
    while current < end:
        month_start = max(current, start)
        if current.month == 12:
            next_month = current.replace(year=current.year + 1, month=1)
        else:
            next_month = current.replace(month=current.month + 1)
        month_end = min(next_month, end)
        
        months.append((month_start.strftime("%Y-%m-%d"), month_end.strftime("%Y-%m-%d")))
        current = next_month
    
    return months


def get_day_ranges_between(start_date: str, end_date: str, chunk_days: int = 3) -> list:
    """
    Generate list of (start_date, end_date) tuples for each N-day chunk between two dates.
    
    Args:
        start_date: Start date 'YYYY-MM-DD'
        end_date: End date 'YYYY-MM-DD' (exclusive)
        chunk_days: Number of days per chunk (default: 3)
    
    Returns:
        List of tuples for each N-day period in the range
    """
    from datetime import timedelta
    
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    chunks = []
    current = start
    
    while current < end:
        chunk_start = current
        chunk_end = min(current + timedelta(days=chunk_days), end)
        
        chunks.append((chunk_start.strftime("%Y-%m-%d"), chunk_end.strftime("%Y-%m-%d")))
        current = chunk_end
    
    return chunks


def process_chunk(chunk_start: str, chunk_end: str, table_name: str, chunk_idx: int, total_chunks: int, append: bool):
    """
    Process a single chunk of data.
    
    Args:
        chunk_start: Start date for this chunk
        chunk_end: End date for this chunk
        table_name: Name of the BigQuery table
        chunk_idx: Index of this chunk (0-based)
        total_chunks: Total number of chunks
        append: Whether to append or create/replace table
    
    Returns:
        Tuple of (chunk_idx, success, error_message)
    """
    try:
        print(f"\n--- Chunk {chunk_idx + 1}/{total_chunks}: {chunk_start} to {chunk_end} ---")
        
        query_verified_attacks_to_bigquery(
            start_date=chunk_start,
            end_date=chunk_end,
            table_name=table_name,
            append=append
        )
        
        return (chunk_idx, True, None)
    except Exception as e:
        error_msg = str(e)
        print(f"  ❌ ERROR in chunk {chunk_idx + 1}: {error_msg}")
        return (chunk_idx, False, error_msg)


def run_batch_pipeline(
    start_date: str,
    end_date: str,
    table_name: str = "verified_attacks",
    min_victim_transfer_usd: float = 0.0,
    chunk_days: int = 3,
    max_concurrent: int = 5,
    skip_table_creation: bool = False
):
    """
    Run the verified attack detection pipeline in time-based batches with parallel execution.
    All results are appended to a single BigQuery table.
    
    Args:
        start_date: Start date 'YYYY-MM-DD'
        end_date: End date 'YYYY-MM-DD' (exclusive)
        table_name: Name for the BigQuery results table
        min_victim_transfer_usd: Minimum value of victim's prior transfer (0 = all transfers considered)
        chunk_days: Number of days per chunk (default: 3)
        max_concurrent: Maximum number of concurrent queries (default: 5, max recommended: 10)
        skip_table_creation: If True, append to existing table instead of creating new one (default: False)
    
    Returns:
        Full BigQuery table ID
    """
    print("=" * 70)
    print("BATCH PIPELINE: Verified Address Poisoning Detection (PARALLEL)")
    print("=" * 70)
    print(f"Date range: {start_date} to {end_date}")
    print(f"Min victim transfer: ${min_victim_transfer_usd} (0 = all transfers)")
    print(f"Chunk size: {chunk_days} day(s)")
    print(f"Max concurrent queries: {max_concurrent}")
    print(f"Mode: {'APPEND to existing table' if skip_table_creation else 'CREATE new table'}")
    print("=" * 70)
    
    chunks = get_day_ranges_between(start_date, end_date, chunk_days=chunk_days)
    print(f"\nTotal chunks to process: {len(chunks)}")
    for i, (s, e) in enumerate(chunks[:5]):  # Show first 5 chunks
        print(f"  {i+1}. {s} to {e}")
    if len(chunks) > 5:
        print(f"  ... and {len(chunks) - 5} more chunks")
    
    print("\n" + "=" * 70)
    print("PROCESSING CHUNKS IN PARALLEL...")
    print("=" * 70)
    
    dest_table = f"{PROJECT_ID}.{BQ_DATASET}.{table_name}"
    
    successful = 0
    failed = 0
    errors = []
    
    if skip_table_creation:
        # All chunks run in parallel with append=True
        print(f"\n🚀 Appending to existing table with up to {max_concurrent} concurrent queries...")
        
        with ThreadPoolExecutor(max_workers=max_concurrent) as executor:
            # Submit all chunks
            futures = {
                executor.submit(
                    process_chunk, 
                    chunk_start, 
                    chunk_end, 
                    table_name, 
                    i,
                    len(chunks),
                    append=True  # All chunks append
                ): (i, chunk_start, chunk_end)
                for i, (chunk_start, chunk_end) in enumerate(chunks)
            }
            
            # Process completed chunks
            for future in tqdm(as_completed(futures), total=len(chunks), desc="Chunks"):
                chunk_idx, chunk_start, chunk_end = futures[future]
                idx, success, error_msg = future.result()
                
                if success:
                    successful += 1
                else:
                    failed += 1
                    errors.append((chunk_start, chunk_end, error_msg))
    else:
        # Process first chunk synchronously to create the table
        if len(chunks) > 0:
            print("\n🔧 Creating table with first chunk...")
            first_start, first_end = chunks[0]
            result = process_chunk(first_start, first_end, table_name, 0, len(chunks), append=False)
            
            if not result[1]:
                print(f"\n❌ Failed to create table with first chunk. Aborting.")
                print(f"Error: {result[2]}")
                return dest_table
            
            print(f"✓ Table created successfully")
            successful = 1
        
        # Process remaining chunks in parallel
        remaining_chunks = chunks[1:]
        
        if len(remaining_chunks) > 0:
            print(f"\n🚀 Processing {len(remaining_chunks)} remaining chunks with up to {max_concurrent} concurrent queries...")
            
            with ThreadPoolExecutor(max_workers=max_concurrent) as executor:
                # Submit all chunks
                futures = {
                    executor.submit(
                        process_chunk, 
                        chunk_start, 
                        chunk_end, 
                        table_name, 
                        i + 1,  # +1 because we already processed chunk 0
                        len(chunks),
                        append=True  # Remaining chunks append
                    ): (i + 1, chunk_start, chunk_end)
                    for i, (chunk_start, chunk_end) in enumerate(remaining_chunks)
                }
                
                # Process completed chunks
                for future in tqdm(as_completed(futures), total=len(remaining_chunks), desc="Chunks"):
                    chunk_idx, chunk_start, chunk_end = futures[future]
                    idx, success, error_msg = future.result()
                    
                    if success:
                        successful += 1
                    else:
                        failed += 1
                        errors.append((chunk_start, chunk_end, error_msg))
    
    # Get final count
    try:
        count = client.query(f"SELECT COUNT(*) as cnt FROM `{dest_table}`").to_dataframe()['cnt'].iloc[0]
        total_attacks = count
    except:
        total_attacks = "Unknown"
    
    print("\n" + "=" * 70)
    print("BATCH PIPELINE COMPLETE")
    print("=" * 70)
    print(f"✓ Successful chunks: {successful}/{len(chunks)}")
    if failed > 0:
        print(f"❌ Failed chunks: {failed}/{len(chunks)}")
        print(f"\nFailed chunks:")
        for start, end, error in errors[:10]:  # Show first 10 errors
            print(f"  - {start} to {end}: {error[:100]}")
        if len(errors) > 10:
            print(f"  ... and {len(errors) - 10} more errors")
    print(f"✓ Total verified attacks: {total_attacks:,}" if isinstance(total_attacks, int) else f"✓ Total verified attacks: {total_attacks}")
    print(f"✓ BigQuery table: {dest_table}")
    print("=" * 70)
    
    return dest_table


# ----------------------------- CONFIGURATION ---------------------------------
PIPELINE_START_DATE = "2025-01-01"
PIPELINE_END_DATE = "2025-02-01"  # Exclusive
TABLE_NAME = "arbitrum_attacks"
CHUNK_DAYS = 1  # Process data in 3-day chunks
MAX_CONCURRENT = 10  # Maximum concurrent queries (5-10 recommended to avoid rate limits)
SKIP_TABLE_CREATION = False  # Set to True to append to existing table instead of creating new
# -----------------------------------------------------------------------------

# Run the batch pipeline
verified_attacks_table = run_batch_pipeline(
    start_date=PIPELINE_START_DATE,
    end_date=PIPELINE_END_DATE,
    table_name=TABLE_NAME,
    chunk_days=CHUNK_DAYS,
    max_concurrent=MAX_CONCURRENT,
    skip_table_creation=SKIP_TABLE_CREATION
)

BATCH PIPELINE: Verified Address Poisoning Detection (PARALLEL)
Date range: 2025-01-01 to 2025-02-01
Min victim transfer: $0.0 (0 = all transfers)
Chunk size: 1 day(s)
Max concurrent queries: 10
Mode: CREATE new table

Total chunks to process: 31
  1. 2025-01-01 to 2025-01-02
  2. 2025-01-02 to 2025-01-03
  3. 2025-01-03 to 2025-01-04
  4. 2025-01-04 to 2025-01-05
  5. 2025-01-05 to 2025-01-06
  ... and 26 more chunks

PROCESSING CHUNKS IN PARALLEL...

🔧 Creating table with first chunk...

--- Chunk 1/31: 2025-01-01 to 2025-01-02 ---
VERIFIED ATTACK DETECTION (with prefix/suffix matching)
Date range: 2025-01-01 to 2025-01-02
Deduplication: Closest temporal match (most recent before attack)
Filters: Removes perfect matches (full 40-char address match)
Data source: Arbitrum logs table
📁 Creating dataset: 1044108980210.arbitrum_vlad_query
✓ Created dataset: 1044108980210.arbitrum_vlad_query

🔍 Running verified attack detection...
📝 Destination: 1044108980210.arbitrum_vlad_query.arbitrum_a